In [ ]:
%pip install pandas

# Projeto Probest - Parte 1: Análise Exploratória (EDA)

Iniciando a Etapa 1 do projeto: Análise Exploratória de Dados (EDA).

In [14]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 1. Carregamento dos Dados

Carregando o arquivo `ndt_tests_corrigido.csv` e verificando sua estrutura.

In [7]:
file_path = 'ndt_tests_corrigido.csv'

try:
    df = pd.read_csv(file_path)
    print(f"Dataset carregado: {file_path}")
    print(f"Formato: {df.shape}")
    
    print("\nColunas disponíveis:")
    print(df.columns.tolist())
    
    print("\nVerificando dados (primeiras linhas):")
    print(df.head())
    
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em '{file_path}'")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dataset carregado: ndt_tests_corrigido.csv
Formato: (7087, 8)

Colunas disponíveis:
['timestamp', 'download_throughput_bps', 'rtt_download_sec', 'upload_throughput_bps', 'rtt_upload_sec', 'packet_loss_percent', 'client', 'server']

Verificando dados (primeiras linhas):
                       timestamp  download_throughput_bps  rtt_download_sec  upload_throughput_bps  rtt_upload_sec  packet_loss_percent    client    server
0  2025-08-09 15:28:02.000 +0000             8.223657e+07          0.231862           6.973217e+07        0.247727             0.000000  client12  server06
1  2025-08-09 15:30:11.000 +0000             9.027315e+08          0.012000           8.521780e+08        0.005423             0.008226  client01  server07
2  2025-08-10 04:27:43.000 +0000             5.910655e+08          0.014000           2.812188e+08        0.014544             5.954284  client13  server07
3  2025-08-09 22:45:07.000 +0000             6.721139e+08          0.011000           1.135400e+08        

## 2. Pré-processamento: Normalização

A coluna `packet_loss_percent` está em formato percentual (0-100). Para a modelagem (especialmente Beta-Binomial), precisamos dela como uma fração (0-1). Vamos criar uma nova coluna `packet_loss_fraction`.

In [12]:
# Criar a coluna de fração de perda dividindo por 100
if 'packet_loss_percent' in df.columns:
    df['packet_loss_fraction'] = df['packet_loss_percent'] / 100
    print("Coluna 'packet_loss_fraction' criada.")
    
    # Verificar a nova coluna
    print(df[['packet_loss_percent', 'packet_loss_fraction']].head())
else:
    print("AVISO: Coluna 'packet_loss_percent' não encontrada.")

Coluna 'packet_loss_fraction' criada.
   packet_loss_percent  packet_loss_fraction
0             0.000000              0.000000
1             0.008226              0.000082
2             5.954284              0.059543
3             0.261528              0.002615
4             1.381646              0.013816


## 3. Definição das Estatísticas e Variáveis (Versão Corrigida)

Definindo as variáveis de interesse (com os nomes corretos do CSV) e as funções de agregação para as estatísticas descritivas.

In [20]:
variables_to_analyze = [
    'download_throughput_bps',
    'upload_throughput_bps',
    'rtt_download_sec',
    'rtt_upload_sec',
    'packet_loss_fraction'
]

group_by_client_col = 'client'
group_by_server_col = 'server'

def q_0_9(x):
    return x.quantile(0.9)

def q_0_99(x):
    return x.quantile(0.99)

stats_to_calculate = ['mean', 'median', 'var', 'std', q_0_9, q_0_99]

print("Variáveis e estatísticas (corrigidas) definidas.")

Variáveis e estatísticas (corrigidas) definidas.


## 4. Análise Agrupada por Cliente

Cálculo das estatísticas descritivas para cada variável, agrupadas por `client`.

In [22]:
try:
    stats_by_client = df.groupby(group_by_client_col)[variables_to_analyze].agg(stats_to_calculate)
    
    stats_by_client.rename(columns={'std': 'std_dev'}, level=1, inplace=True)

    print("--- Estatísticas por Cliente ---")
    print(stats_by_client)

except KeyError as e:
    print(f"ERRO: Chave não encontrada. Verifique os nomes das colunas. Detalhe: {e}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

--- Estatísticas por Cliente ---
         download_throughput_bps                                                                       upload_throughput_bps                                                                       rtt_download_sec                                                   rtt_upload_sec                                                   packet_loss_fraction                                                  
                            mean        median           var       std_dev         q_0_9        q_0_99                  mean        median           var       std_dev         q_0_9        q_0_99             mean    median       var   std_dev     q_0_9    q_0_99           mean    median       var   std_dev     q_0_9    q_0_99                 mean    median       var   std_dev     q_0_9    q_0_99
client                                                                                                                                                                     

## 5. Análise Agrupada por Servidor

Cálculo das estatísticas descritivas para cada variável, agrupadas por `server`.

In [23]:
try:
    stats_by_server = df.groupby(group_by_server_col)[variables_to_analyze].agg(stats_to_calculate)
    
    stats_by_server.rename(columns={'std': 'std_dev'}, level=1, inplace=True)

    print("--- Estatísticas por Servidor ---")
    print(stats_by_server)

except KeyError as e:
    print(f"ERRO: Chave não encontrada. Verifique os nomes das colunas. Detalhe: {e}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

--- Estatísticas por Servidor ---
         download_throughput_bps                                                                       upload_throughput_bps                                                                       rtt_download_sec                                                   rtt_upload_sec                                                   packet_loss_fraction                                                  
                            mean        median           var       std_dev         q_0_9        q_0_99                  mean        median           var       std_dev         q_0_9        q_0_99             mean    median       var   std_dev     q_0_9    q_0_99           mean    median       var   std_dev     q_0_9    q_0_99                 mean    median       var   std_dev     q_0_9    q_0_99
server                                                                                                                                                                    

## Próximos Passos (Início da Etapa 2)

A Etapa 1 está concluída. As tabelas de resumo acima são a base para a próxima fase.

**Para a Etapa 2**, precisamos selecionar duas entidades (dois clientes, ou um cliente e um servidor) que pareçam interessantes ou distintas para a análise gráfica.

Analise os outputs acima e me diga quais entidades você quer selecionar.